In [1]:
# PyTorch基础学习笔记

# 1. Tensor操作
import torch

# 创建Tensor
x = torch.tensor([1, 2, 3])
print(x)

# 基本运算
y = torch.tensor([4, 5, 6])
print(x + y)       # 加法
print(x * y)       # 乘法
print(x @ y)       # 点积（向量点积）

# 索引、切片、形状变化
z = torch.arange(9).reshape(3, 3)
print(z)
print(z[1, :])     # 第2行
print(z[:, 2])     # 第3列
print(z.view(9))   # 拉平成一维


tensor([1, 2, 3])
tensor([5, 7, 9])
tensor([ 4, 10, 18])
tensor(32)
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([3, 4, 5])
tensor([2, 5, 8])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])


In [2]:
# 使用GPU加速（如果有GPU）
if torch.cuda.is_available():
    x = x.to('cuda')
    print("Tensor moved to GPU")

In [3]:
# 2. 自动微分（Autograd）

# 创建带梯度的Tensor
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a * b
print(c)

# 反向传播计算梯度
c.backward()
print(a.grad)  # dc/da = b = 3
print(b.grad)  # dc/db = a = 2

tensor(6., grad_fn=<MulBackward0>)
tensor(3.)
tensor(2.)


In [4]:
# 3. 神经网络模块（nn.Module）
import torch.nn as nn

# 定义一个简单的两层神经网络
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)


In [5]:
# 4. 损失函数

# 均方误差损失
mse_loss = nn.MSELoss()
pred = torch.tensor([0.5], requires_grad=True)
target = torch.tensor([1.0])
loss = mse_loss(pred, target)
print(loss)

# 交叉熵损失（分类用）
ce_loss = nn.CrossEntropyLoss()
preds = torch.randn(3, 5)  # batch=3, class=5
targets = torch.tensor([1, 0, 4])
loss = ce_loss(preds, targets)
print(loss)

tensor(0.2500, grad_fn=<MseLossBackward0>)
tensor(2.0038)


In [6]:
# 5. 优化器
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练一步示例
input_sample = torch.randn(1, 2)
output = model(input_sample)
target = torch.randn(1, 1)

loss = mse_loss(output, target)
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [7]:
# 6. 自定义训练循环

# 准备数据
X = torch.randn(100, 2)
Y = torch.randn(100, 1)

# 训练模型
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(X)
    loss = mse_loss(outputs, Y)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.9978
Epoch [2/5], Loss: 0.9937
Epoch [3/5], Loss: 0.9897
Epoch [4/5], Loss: 0.9860
Epoch [5/5], Loss: 0.9825


In [8]:
# 7. Dataset 和 DataLoader
from torch.utils.data import Dataset, DataLoader

# 自定义小数据集
class MyDataset(Dataset):
    def __init__(self):
        self.X = torch.randn(50, 2)
        self.Y = torch.randn(50, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = MyDataset()
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# 遍历DataLoader
for batch_X, batch_Y in train_loader:
    outputs = model(batch_X)
    loss = mse_loss(outputs, batch_Y)
    print("Batch Loss:", loss.item())
    break  # 这里只看一个batch示例

Batch Loss: 0.9549804925918579


In [10]:
# 8. 保存与加载模型

# 保存模型参数
torch.save(model.state_dict(), "simple_model.pth", weights_only=True)

# 加载模型参数
model2 = SimpleNN()
model2.load_state_dict(torch.load("simple_model.pth"))
model2.eval()
print("模型加载完成")

TypeError: save() got an unexpected keyword argument 'weights_only'